In [6]:
import numpy as np
import pickle
import numpy as np
import sklearn.metrics as metrics
from sklearn.metrics import accuracy_score
import pickle
import sklearn.metrics as metrics
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn import preprocessing
import pandas as pd
import os
import time, os
import pymysql

- sql database connect

In [18]:
db_dsml = pymysql.connect(
    host = 'localhost', 
    port = 3306, 
    user = 'stock_user', 
    passwd = 'bigdata', 
    db = 'stock', 
    charset = 'utf8'
)

cursor = db_dsml.cursor()

# 데이터셋 생성
- 최종 데이터셋의 코드 (거래대금 10억, 10일치, 종가상승 2%, min-max scaling)

## 과제 1

In [2]:
import pandas as pd
df_stock = pd.read_csv("raw_data.csv") #데이터 불러오기
df_stock['종목코드'] = df_stock['종목코드'].apply(lambda x : str(x).zfill(6))
lst_stock = df_stock.values.tolist()

In [3]:
# dictionary 저장 후 파일 쓰기
dic_code2name = {}
for row in lst_stock:
    name, code, date = row[1], row[2], row[5]
    if date <= '2018-01-01': 
        dic_code2name[code] = name 

In [4]:
OF = open('assignment1_multi_processing.txt', 'w', encoding='UTF-8')
for cd in dic_code2name: 
    OF.write(f'{cd}\t{dic_code2name[cd]}\n')

## train dataset

In [40]:
dic_code2date = {}
for code in tqdm(dic_code2name.keys()):
    sql_query = '''
                SELECT *
                FROM stock_{}
                WHERE Date BETWEEN '2018-01-01' AND '2020-12-31'
                '''.format(code)
    stock = pd.read_sql(sql = sql_query, con = db_dsml)
    lst_stock = stock.values.tolist()
    
    for row in lst_stock:
        if row[4] * row[5] >= 10000000000:
            data = '{}\t{}\n'.format(code, row[0].strftime('%Y%m%d'))
            if code not in dic_code2date.keys():
                dic_code2date[code] = []
                dic_code2date[code].append(row[0].strftime('%Y%m%d'))
            else:
                dic_code2date[code].append(row[0].strftime('%Y%m%d'))

100%|███████████████████████████████████████| 1999/1999 [00:24<00:00, 80.55it/s]


In [41]:
OF = open('assignment3-1_100_10_y.txt', 'w', encoding = 'utf-8')
for code in tqdm(dic_code2date.keys()):
    sql_query = '''
                SELECT *
                FROM stock_{}
                WHERE (Date BETWEEN '2018-01-01' AND '2020-12-31')  AND (Open not in (0))
                '''.format(code)
    
    
    stock = pd.read_sql(sql = sql_query, con = db_dsml)
    lst_stock = stock.values.tolist()
    
    
    ##### MinMaxScaling 
    stock['Trading'] = stock['Close'] * stock['Volume']
    scaler = MinMaxScaler()
    scale_stock = scaler.fit_transform(stock[['Open', 'High', 'Low', 'Close', 'Trading']])
    df_stock_scale = pd.DataFrame(scale_stock, columns=['Open', 'High', 'Low', 'Close', 'Trading'])
    lst_stock_scale = df_stock_scale.values.tolist()
    
    
    for i, row in enumerate(lst_stock):
               
        if (i < 9) or (i >= len(lst_stock)-1):
            continue

        date = row[0].date().strftime('%Y%m%d')
        
        
        if date not in dic_code2date[code]:
            continue

        sub_stock = lst_stock_scale[i-9:i+1]
        result=[]
        for j, row2 in enumerate(sub_stock): 
            open_, high, low, close, trading_value = row2[:6]
            result+=[open_, high, low, close, trading_value]
        result=','.join(map(str,result))
        
        ratio = int(lst_stock[i+1][-1]>=0.02)
        
        OF.write('\t'.join(map(str,[code,date,result,ratio])) + '\n')              

OF.close()

100%|███████████████████████████████████████| 1790/1790 [00:32<00:00, 54.54it/s]


## validation dataset

In [38]:
dic_code2date_test = {}

for code in tqdm(dic_code2name.keys()):
    sql_query = '''
                SELECT *
                FROM stock_{}
                WHERE (Date BETWEEN '2021-01-01' AND '2021-06-31') AND (Open not in (0))
                '''.format(code)
    stock = pd.read_sql(sql = sql_query, con = db_dsml)
    lst_stock = stock.values.tolist()
    
    for row in lst_stock:
        if row[4] * row[5] >= 10000000000:
            data = '{}\t{}\n'.format(code, row[0].strftime('%Y%m%d'))
            if code not in dic_code2date_test.keys():
                dic_code2date_test[code] = []
                dic_code2date_test[code].append(row[0].strftime('%Y%m%d'))
            else:
                dic_code2date_test[code].append(row[0].strftime('%Y%m%d'))

100%|██████████████████████████████████████| 1999/1999 [00:06<00:00, 298.88it/s]


In [39]:
OF = open('assignment3-2_100_10_y.txt', 'w', encoding = 'utf-8')
for code in tqdm(dic_code2date_test.keys()):
    sql_query = '''
                SELECT *
                FROM stock_{}
                WHERE (Date BETWEEN '2021-01-01' AND '2021-06-31')  AND (Open not in (0))
                '''.format(code)
    stock = pd.read_sql(sql = sql_query, con = db_dsml)
    lst_stock = stock.values.tolist()

    ##### MinMaxScaling 
    stock['Trading'] = stock['Close'] * stock['Volume']
    scaler = MinMaxScaler()
    scale_stock = scaler.fit_transform(stock[['Open', 'High', 'Low', 'Close', 'Trading']])
    df_stock_scale = pd.DataFrame(scale_stock, columns=['Open', 'High', 'Low', 'Close', 'Trading'])
    lst_stock_scale = df_stock_scale.values.tolist()
        
    
    for i, row in enumerate(lst_stock):
               
        if (i < 9) or (i >= len(lst_stock)-1):
            continue

        date = row[0].date().strftime('%Y%m%d')
        
        
        if date not in dic_code2date_test[code]:
            continue

        sub_stock = lst_stock_scale[i-9:i+1]
        result=[]
        for j, row2 in enumerate(sub_stock):
            open_, high, low, close, trading_value = row2[:6]

            result+=[open_, high, low, close, trading_value]
        result=','.join(map(str,result))
        ratio = int(lst_stock[i+1][-1]>=0.02)
        OF.write('\t'.join(map(str,[code,date,result,ratio])) + '\n')              

OF.close()

100%|██████████████████████████████████████| 1634/1634 [00:10<00:00, 162.30it/s]


##  private test dataset

In [ ]:
# (20210101~20210631)

In [44]:
dic_code2date_test = {}

for code in tqdm(dic_code2name.keys()):
    sql_query = '''
                SELECT *
                FROM stock_{}
                WHERE (Date BETWEEN '2021-07-01' AND '2021-12-31') AND (Open not in (0))
                '''.format(code)
    stock = pd.read_sql(sql = sql_query, con = db_dsml)
    lst_stock = stock.values.tolist()
    
    for row in lst_stock:
        if row[4] * row[5] >= 1000000000:
            data = '{}\t{}\n'.format(code, row[0].strftime('%Y%m%d'))
            if code not in dic_code2date_test.keys():
                dic_code2date_test[code] = []
                dic_code2date_test[code].append(row[0].strftime('%Y%m%d'))
            else:
                dic_code2date_test[code].append(row[0].strftime('%Y%m%d'))

100%|██████████████████████████████████████| 1999/1999 [00:07<00:00, 262.85it/s]


In [45]:
OF = open('assignment3-2_private.txt', 'w', encoding = 'utf-8')
for code in tqdm(dic_code2date_test.keys()):
    sql_query = '''
                SELECT *
                FROM stock_{}
                WHERE (Date BETWEEN '2021-07-01' AND '2021-12-31')  AND (Open not in (0))
                '''.format(code)
    stock = pd.read_sql(sql = sql_query, con = db_dsml)
    lst_stock = stock.values.tolist()

    ##### MinMaxScaling 
    stock['Trading'] = stock['Close'] * stock['Volume']
    scaler = MinMaxScaler()
    scale_stock = scaler.fit_transform(stock[['Open', 'High', 'Low', 'Close', 'Trading']])
    df_stock_scale = pd.DataFrame(scale_stock, columns=['Open', 'High', 'Low', 'Close', 'Trading'])
    lst_stock_scale = df_stock_scale.values.tolist()
        
    
    for i, row in enumerate(lst_stock):
               
        if (i < 9) or (i >= len(lst_stock)-1):
            continue

        date = row[0].date().strftime('%Y%m%d')
        
        
        if date not in dic_code2date_test[code]:
            continue

        sub_stock = lst_stock_scale[i-9:i+1]
        result=[]
        for j, row2 in enumerate(sub_stock):
            open_, high, low, close, trading_value = row2[:6]

            result+=[open_, high, low, close, trading_value]
        result=','.join(map(str,result))
        ratio = int(lst_stock[i+1][-1]>=0.02)
        OF.write('\t'.join(map(str,[code,date,result,ratio])) + '\n')              

OF.close()

100%|██████████████████████████████████████| 1812/1812 [00:13<00:00, 134.28it/s]


--------------------

# 모델 선택

In [54]:
IF=open("assignment3-1.txt",'r')
lst_code_date_train=[]
trainX=[]
trainY=[]
for line in IF:
    code, date, x, y = line.strip().split("\t")
    lst_code_date_train.append([code, date])
    trainX.append(list(map(int, x.split(","))))
    trainY.append(int(y))
trainX=np.array(trainX)
trainY=np.array(trainY)

In [55]:
IF=open("assignment3-2.txt",'r')
lst_code_date_test=[]
testX=[]
testY=[]
for line in IF:
    code, date, x, y = line.strip().split("\t")
    lst_code_date_test.append([code, date])
    testX.append(list(map(float, x.split(","))))
    testY.append(int(y))
testX=np.array(testX)
testY=np.array(testY)

## Logistic Regression

In [29]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(max_iter=1000)
lr.fit(trainX, trainY)
predY = lr.predict(testX)
predY_prob = lr.predict_proba(testX)
print("Logistic Regression:", accuracy_score(testY, predY))

Logistic Regression: 0.7846624835542961


In [30]:
for y in predY_prob:
    if y[1] > 0.7: # 없음 
        print(y)        

## Random forest

In [35]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
model.fit(trainX, trainY)
predY = model.predict(testX)
predY_prob = model.predict_proba(testX)
print("Logistic Regression:", accuracy_score(testY, predY))

Logistic Regression: 0.7822082785143204


In [36]:
for y in predY_prob:
    if y[1] > 0.7: 
        print(y)     

[0.27 0.73]


## K-nearest neighbor

In [33]:
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier()
model.fit(trainX, trainY)
predY = model.predict(testX)
predY_prob = model.predict_proba(testX)
print("K nearest neighbor:", accuracy_score(testY, predY))

K nearest neighbor: 0.7419795567250278


In [34]:
for y in predY_prob:
    if y[1] > 0.7: 
        print(y)  

[0.2 0.8]
[0.2 0.8]
[0.2 0.8]
[0.2 0.8]
[0.2 0.8]
[0.2 0.8]
[0.2 0.8]
[0.2 0.8]
[0.2 0.8]
[0.2 0.8]
[0.2 0.8]
[0.2 0.8]
[0.2 0.8]
[0.2 0.8]
[0.2 0.8]
[0.2 0.8]
[0.2 0.8]
[0.2 0.8]
[0.2 0.8]
[0.2 0.8]
[0. 1.]
[0.2 0.8]
[0.2 0.8]
[0.2 0.8]
[0.2 0.8]
[0.2 0.8]
[0.2 0.8]
[0.2 0.8]
[0.2 0.8]
[0.2 0.8]
[0.2 0.8]
[0.2 0.8]
[0.2 0.8]
[0.2 0.8]
[0.2 0.8]
[0. 1.]
[0. 1.]
[0.2 0.8]
[0.2 0.8]
[0.2 0.8]
[0.2 0.8]
[0.2 0.8]
[0.2 0.8]
[0.2 0.8]
[0. 1.]
[0.2 0.8]
[0.2 0.8]
[0.2 0.8]
[0. 1.]
[0.2 0.8]
[0.2 0.8]
[0. 1.]
[0.2 0.8]
[0.2 0.8]
[0.2 0.8]
[0.2 0.8]
[0.2 0.8]
[0.2 0.8]
[0.2 0.8]
[0.2 0.8]
[0.2 0.8]
[0.2 0.8]
[0.2 0.8]
[0.2 0.8]
[0.2 0.8]
[0.2 0.8]
[0.2 0.8]
[0.2 0.8]
[0.2 0.8]
[0.2 0.8]
[0.2 0.8]
[0.2 0.8]
[0.2 0.8]
[0.2 0.8]
[0.2 0.8]
[0.2 0.8]
[0.2 0.8]
[0.2 0.8]
[0. 1.]
[0.2 0.8]
[0.2 0.8]
[0.2 0.8]
[0.2 0.8]
[0.2 0.8]
[0.2 0.8]
[0. 1.]
[0.2 0.8]
[0.2 0.8]
[0.2 0.8]
[0.2 0.8]
[0.2 0.8]
[0.2 0.8]
[0.2 0.8]
[0.2 0.8]
[0.2 0.8]
[0.2 0.8]
[0.2 0.8]
[0.2 0.8]
[0.2 0.8]
[0. 1.]
[0.2 0.8]
[0.2 0.8

## XGBoost 

In [56]:
from xgboost import XGBClassifier

xgb = XGBClassifier()
xgb.fit(trainX, trainY)
predY_prob = xgb.predict_proba(testX)
predY = xgb.predict(testX)
accuracy = accuracy_score(testY, predY)
print("xgb: ",accuracy)

/home/sookim06/.local/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[21:16:19] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
xgb:  0.7819046655196843


In [57]:
for y in predY_prob: 
    if y[1] >= 0.7: 
        print(y)

[0.2782777 0.7217223]
[0.25654137 0.7434586 ]
[0.27427334 0.72572666]
[0.24586576 0.75413424]
[0.13825965 0.86174035]
[0.20731932 0.7926807 ]
[0.28832793 0.71167207]
[0.26936668 0.7306333 ]
[0.27497894 0.72502106]
[0.29392266 0.70607734]
[0.295911 0.704089]
[0.2712589 0.7287411]
[0.25475442 0.7452456 ]
[0.12325442 0.8767456 ]
[0.2958045 0.7041955]
[0.18597966 0.81402034]
[0.29437393 0.7056261 ]
[0.29261845 0.70738155]


## Deep Learning

In [37]:
from tqdm.notebook import tqdm
from tensorflow import keras

model = keras.models.Sequential()
model.add(keras.layers.InputLayer(input_shape=(trainX.shape[1],)))
model.add(keras.layers.Dense(128, activation='hard_sigmoid'))
model.add(keras.layers.Dense(64, activation='hard_sigmoid'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(64, activation='hard_sigmoid'))
model.add(keras.layers.Dense(32, activation='hard_sigmoid'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(1, activation='sigmoid'))
    
model.compile(optimizer=keras.optimizers.Adam(
    learning_rate=keras.optimizers.schedules.ExponentialDecay(0.01,decay_steps=100000,decay_rate=0.96)), 
    loss="binary_crossentropy",
    metrics=['accuracy'])
model.summary()

2022-01-28 20:49:31.570713: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               6528      
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 64)                4160      
                                                                 
 dense_3 (Dense)             (None, 32)                2080      
                                                                 
 dropout_1 (Dropout)         (None, 32)                0         
                                                                 
 dense_4 (Dense)             (None, 1)                 3

2022-01-28 20:49:31.604382: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-28 20:49:31.605237: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-28 20:49:31.606629: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-01-28 20:49:31.652768: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

In [38]:
es = keras.callbacks.EarlyStopping(monitor="val_accuracy", patience=20, verbose=1, mode="auto", restore_best_weights=True)
lr = keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=10, verbose=1)
sv = keras.callbacks.ModelCheckpoint(
    "./model_keras_mlp.h5", monitor='val_accuracy', verbose=1, save_best_only=True,
    save_weights_only=False, mode='auto', save_freq='epoch',
    options=None
    )

history = model.fit(trainX, trainY, validation_split=0.20, batch_size=128, epochs=50, callbacks=[es, sv], shuffle=True)            

Epoch 1/50
 63/805 [=>............................] - ETA: 1s - loss: 0.5287 - accuracy: 0.7839

2022-01-28 20:49:34.662649: I tensorflow/stream_executor/cuda/cuda_blas.cc:1774] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


805/805 [==============================] - ETA: 0s - loss: 0.5190 - accuracy: 0.7866
Epoch 00001: val_accuracy improved from -inf to 0.77068, saving model to ./model_keras_mlp.h5
805/805 [==============================] - 3s 3ms/step - loss: 0.5190 - accuracy: 0.7866 - val_loss: 0.5372 - val_accuracy: 0.7707
Epoch 2/50
797/805 [============================>.] - ETA: 0s - loss: 0.5170 - accuracy: 0.7864
Epoch 00002: val_accuracy did not improve from 0.77068
805/805 [==============================] - 2s 2ms/step - loss: 0.5165 - accuracy: 0.7868 - val_loss: 0.5427 - val_accuracy: 0.7707
Epoch 3/50
789/805 [============================>.] - ETA: 0s - loss: 0.5157 - accuracy: 0.7869
Epoch 00003: val_accuracy did not improve from 0.77068
805/805 [==============================] - 2s 2ms/step - loss: 0.5159 - accuracy: 0.7868 - val_loss: 0.5370 - val_accuracy: 0.7707
Epoch 4/50
793/805 [============================>.] - ETA: 0s - loss: 0.5159 - accuracy: 0.7868
Epoch 00004: val_accuracy did 

In [44]:
predY = model.predict(testX)
for y in predY: 
    if y > 0.25:
        print(y) # 모두 0.1~0.25 사이의 값임

## LSTM

In [45]:
import numpy as np
import sklearn.metrics as metrics
import pickle

IF=open("assignment3-1.txt",'r')
lst_code_date=[]
trainX=[]
trainY=[]
for line in IF:
    code, date, x, y = line.strip().split("\t")
    lst_code_date.append([code, date])
    trainX.append([])
    tmpx = list(map(float, x.split(",")))
    for i in range(0,len(tmpx),5):
        trainX[-1].append(tmpx[i:i+5])
    trainY.append(int(y))
trainX=np.array(trainX)
trainY=np.array(trainY)

In [46]:
import numpy as np
import sklearn.metrics as metrics
import pickle
from sklearn import svm
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

IF=open("assignment3-2.txt",'r')
lst_code_date=[]
testX=[]
testY=[]
for line in IF:
    code, date, x, y = line.strip().split("\t")
    lst_code_date.append([code, date])
    testX.append([])
    tmpx = list(map(float, x.split(",")))
    for i in range(0,len(tmpx),5):
        testX[-1].append(tmpx[i:i+5])    
    testY.append(int(y))
testX=np.array(testX)
testY=np.array(testY)

In [48]:
from tqdm.notebook import tqdm

from tensorflow import keras

model = keras.models.Sequential()
model.add(keras.layers.InputLayer(input_shape=(trainX.shape[1],trainX.shape[2])))
model.add(keras.layers.LSTM(128, activation='relu', return_sequences=True))
model.add(keras.layers.LSTM(64, activation='relu', return_sequences=True))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.LSTM(64, activation='relu', return_sequences=True))
model.add(keras.layers.LSTM(32, activation='relu', return_sequences=False))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(1, activation='sigmoid'))
    
model.compile(optimizer=keras.optimizers.Adam(
    learning_rate=keras.optimizers.schedules.ExponentialDecay(0.05,decay_steps=100000,decay_rate=0.96)), 
    loss="binary_crossentropy",
    metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 10, 128)           68608     
                                                                 
 lstm_1 (LSTM)               (None, 10, 64)            49408     
                                                                 
 dropout_2 (Dropout)         (None, 10, 64)            0         
                                                                 
 lstm_2 (LSTM)               (None, 10, 64)            33024     
                                                                 
 lstm_3 (LSTM)               (None, 32)                12416     
                                                                 
 dropout_3 (Dropout)         (None, 32)                0         
                                                                 
 dense_5 (Dense)             (None, 1)                

In [ ]:
es = keras.callbacks.EarlyStopping(monitor="val_accuracy", patience=20, verbose=1, mode="auto", restore_best_weights=True)
lr = keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=10, verbose=1)
sv = keras.callbacks.ModelCheckpoint(
    "./model_keras_lstm.h5", monitor='val_accuracy', verbose=1, save_best_only=True,
    save_weights_only=False, mode='auto', save_freq='epoch',
    options=None
    )

history = model.fit(trainX, trainY, validation_split=0.20, batch_size=128, epochs=5, callbacks=[es, sv], shuffle=True)            

In [52]:
predY = model.predict(testX)
predY # 결과 값으로 모두 같은 값이 나옴 

array([[0.20312916],
       [0.20312916],
       [0.20312916],
       ...,
       [0.20312916],
       [0.20312916],
       [0.20312916]], dtype=float32)

------------

- __정확도 0.78 이상:__ Logistic Regression, Random forest, XGBoost, Deep Learning, LSTM 
    - K-nearest neighbor 을 제외한 모든 모델이 0.78xxx 로 비슷한 정확도를 가지고 있음
- __DL, LSTM은 0.7868로 정확도가 가장 높지만 신뢰성이 없음__  
    - DL: 예측 값이 1~0.25 사이임 -> 모두 0으로 예측 
    - LSTM: 예측 값이 0.20312916 -> 모두 같은 값으로 예측하며, 모두 0으로 예측 
- __Logistic Regression, Random forest, XGBoost__
    - model.predict_proba로 얻어낸 0, 1의 예측 확률을 보았을 때 XGBoost 모델이 가장 타당할 것으로 판단
- __최종 모델 선택__
    - XGBoost

------------

- __장부생성, 수익률 계산 함수__

In [1]:
def make_assignment4(trading_value, days, scaling='n'):
    lst_output=[]
    for (code, date), y in zip(lst_code_date_test, predY_prob):
        if y[1] >= 0.9: 
            lst_output.append([code, date, "buy", "all"])  
            lst_output.append([code, date+"n", "sell", "all"])  
        elif y[1] >= 0.7: 
            lst_output.append([code, date, "buy", "r40"])  
            lst_output.append([code, date+"n", "sell", "all"]) 
        elif y[1] >= 0.6: 
            lst_output.append([code, date, "buy", "r20"])  
            lst_output.append([code, date+"n", "sell", "all"])  
        elif y[1] >= 0.5: 
            lst_output.append([code, date, "buy", "r10"])  
            lst_output.append([code, date+"n", "sell", "all"]) 



    lst_output.sort(key=lambda x:x[1]) 
    OF=open("assignment4_xgb_{}_{}_{}.txt".format(trading_value, days, scaling),'w') 
    for row in lst_output:
        OF.write("\t".join(map(str, row))+"\n")
    OF.close()

In [2]:
def make_rate(trading_value, days, scaling='n'):
    start_money = 10000000 # 초기 현금 1천만원
    money = start_money
    dic_code2num ={}  # 보유 종목

    IF=open("assignment4_xgb_{}_{}_{}.txt".format(trading_value, days, scaling),'r')
    for i, line in enumerate(tqdm(IF)): #주문 일지를 한 줄 읽어 옴
        code, date, request, amount = line.strip().split("\t")

        sql_query = '''
                    SELECT *
                    FROM stock_{}
                    WHERE Date BETWEEN '2021-01-01' AND '2021-06-31'
                    '''.format(code)
        stock = pd.read_sql(sql = sql_query, con = db_dsml)
        lst_data = stock.values.tolist()    

        ##############################################################################################
        for idx, row in enumerate(lst_data):
            data_date = row[0].date().strftime('%Y%m%d')


            if 'n' in date:
                date_n = date[:-1]
                if data_date == date_n: 
                    close = lst_data[idx+1][-3] 
                    break            

            else: 
                if data_date == date:
                    close = row[-3] 
                    break
        ##############################################################################################

        if request == 'buy': 
            if amount.startswith('r'):
                request_money = money * float(amount.lstrip("r")) / 100
            elif amount == 'all':
                request_money = money
            elif amount.isdigit():
                request_money = int(amount)
            else:
                raise Exception('Not permitted option')
            request_money = min(request_money, money)
            buy_num = int(request_money / close)
            money -= buy_num * close  
            if code not in dic_code2num:
                dic_code2num[code] = 0
            dic_code2num[code] += buy_num 
        if request == 'sell':
            if amount == 'all':
                sell_num = dic_code2num[code]
            else:
                raise Exception('Not permitted option')            
            money += sell_num * close
            dic_code2num[code] -= sell_num
            if dic_code2num[code] == 0:
                del dic_code2num[code]
    IF.close()            

    if dic_code2num != {}: 
        raise Exception('Not empty stock') 

    print("Final earning rate : {} %".format(str((money-start_money) / start_money * 100)))
#     return str((money-start_money) / start_money * 100)

------------

- __train, test dataset 생성 함수__

In [11]:
def make_train(trading_value, days, scaling='n'):
    IF=open("assignment3-1_{}_{}_{}.txt".format(trading_value, days, scaling),'r')
    lst_code_date_train=[]
    trainX=[]
    trainY=[]
    for line in IF:
        code, date, x, y = line.strip().split("\t")
        lst_code_date_train.append([code, date])
        trainX.append(list(map(float, x.split(","))))
        trainY.append(int(y))
    trainX=np.array(trainX)
    trainY=np.array(trainY)
    
    return trainX, trainY

In [15]:
def make_test(trading_value, days, scaling='n'):
    IF=open("assignment3-2_{}_{}_{}.txt".format(trading_value, days, scaling),'r')
    lst_code_date_test=[]
    testX=[]
    testY=[]
    for line in IF:
        code, date, x, y = line.strip().split("\t")
        lst_code_date_test.append([code, date])
        testX.append(list(map(float, x.split(","))))
        testY.append(int(y))
    testX=np.array(testX)
    testY=np.array(testY)
    
    return testX, testY, lst_code_date_test

------------

# XGBoost 모델링

## 1차 모델: xgb (base model) 

### 예측 및 매매

In [82]:
make_assignment4(100, 10)

### 수익률 계산

In [83]:
make_rate(100, 10)

546it [00:02, 256.59it/s]


'35.57367'

- __1차 모델 수익률:__  35.57367 %

--------------

## 2차 모델

- 거래대금 임계값 1000억 100억 10억 

### 예측 및 매매

- __10일 기준 1000억, 100억, 10억 비교__ (데이터 생성 코드 생략)

In [19]:
from xgboost import XGBClassifier

trainX_1000_10, trainY_1000_10  = make_train(1000, 10, 'n')
testX_1000_10, testY_1000_10, lst_code_date_test  = make_test(1000, 10, 'n')

xgb_1000_10 = XGBClassifier()
xgb_1000_10.fit(trainX_1000_10, trainY_1000_10)
predY_prob = xgb_1000_10.predict_proba(testX_1000_10)
predY = xgb_1000_10.predict(testX_1000_10)
accuracy = accuracy_score(testY_1000_10, predY)
print("xgb_1000_10: ",accuracy)

make_assignment4(1000, 10)
make_rate(1000, 10)

/home/sookim06/.local/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[22:55:59] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
xgb_1000_10:  0.7491896272285251


520it [00:02, 211.33it/s]

Final earning rate : -0.7133999999999999 %


In [20]:
trainX_100_10, trainY_100_10 = make_train(100, 10, 'n')
testX_100_10, testY_100_10, lst_code_date_test = make_test(100, 10, 'n')

xgb_100_10 = XGBClassifier()
xgb_100_10.fit(trainX_100_10, trainY_100_10)
predY_prob = xgb_100_10.predict_proba(testX_100_10)
predY = xgb_100_10.predict(testX_100_10)
accuracy = accuracy_score(testY_100_10, predY)
print("xgb_100_10: ",accuracy)

make_assignment4(100, 10)
make_rate(100, 10)

/home/sookim06/.local/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[22:56:17] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
xgb_100_10:  0.7819046655196843


546it [00:02, 228.89it/s]

Final earning rate : 35.57367 %


In [21]:
trainX_10_10, trainY_10_10 = make_train(10, 10, 'n')
testX_10_10, testY_10_10, lst_code_date_test = make_test(10, 10, 'n')

xgb_10_10 = XGBClassifier()
xgb_10_10.fit(trainX_10_10, trainY_10_10)
predY_prob = xgb_10_10.predict_proba(testX_10_10)
predY = xgb_10_10.predict(testX_10_10)
accuracy = accuracy_score(testY_10_10, predY)
print("xgb_10_10: ",accuracy)

make_assignment4(10, 10)
make_rate(10, 10)

/home/sookim06/.local/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[22:56:36] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
xgb_10_10:  0.8019427799571783


704it [00:03, 230.31it/s]

Final earning rate : 54.39008 %


- __2차 모델 수익률:__ 54.39008 %

## 3차 모델

### 예측 및 매매

In [22]:
trainX_10_10, trainY_10_10 = make_train(10, 10, 'y')
testX_10_10, testY_10_10, lst_code_date_test = make_test(10, 10, 'y')

xgb_10_10_sclae = XGBClassifier()
xgb_10_10_sclae.fit(trainX_10_10, trainY_10_10)
predY_prob = xgb_10_10_sclae.predict_proba(testX_10_10)
predY = xgb_10_10_sclae.predict(testX_10_10)
accuracy = accuracy_score(testY_10_10, predY)
print("xgb_10_10_sclae: ",accuracy)

/home/sookim06/.local/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[22:57:30] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
xgb_10_10:  0.7816456150939058


In [23]:
make_assignment4(100, 10)

### 수익률 계산

In [24]:
make_rate(100, 10)

14122it [01:20, 175.48it/s]

Final earning rate : 310.61341 %


- 정확도는 낮아졌지만 수익률이 대폭 상승
- __3차 모델 수익률:__ 310.61341 %

## 4차 모델: xgb_4

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

xgb_model = XGBClassifier(tree_method='gpu_exact') # 코랩 gpu 사용 

# 후보 파라미터 
params = {'max_depth':[4, 5], 'n_estimators':[100, 200, 300, 400, 500, 600], 'learning_rate': [0.1, 0.01]}

gridcv = GridSearchCV(xgb_model, param_grid=params, cv=3)

# 파라미터 튜닝 
gridcv.fit(trainX, trainY, early_stopping_rounds=30, eval_metric='auc', eval_set=[(valX, valY)])

# 최적의 파라미터 
print(gridcv.best_params_)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
[69]	validation_0-auc:0.564724
[70]	validation_0-auc:0.564834
[71]	validation_0-auc:0.565042
[72]	validation_0-auc:0.565203
[73]	validation_0-auc:0.565127
[74]	validation_0-auc:0.565236
[75]	validation_0-auc:0.565295
[76]	validation_0-auc:0.565344
[77]	validation_0-auc:0.565494
[78]	validation_0-auc:0.565513
[79]	validation_0-auc:0.565564
[80]	validation_0-auc:0.566237
[81]	validation_0-auc:0.566218
[82]	validation_0-auc:0.566211
[83]	validation_0-auc:0.566255
[84]	validation_0-auc:0.566371
[85]	validation_0-auc:0.566197
[86]	validation_0-auc:0.566355
[87]	validation_0-auc:0.566565
[88]	validation_0-auc:0.566542
[89]	validation_0-auc:0.566697
[90]	validation_0-auc:0.566838
[91]	validation_0-auc:0.567353
[92]	validation_0-auc:0.56719
[93]	validation_0-auc:0.56726
[94]	validation_0-auc:0.567445
[95]	validation_0-auc:0.5677
[96]	validation_0-auc:0.567759
[97]	validation_0-auc:0.567957
[98]	validation_0-auc:0.567954
[99]	validation_0-auc:0.56798
[100]	va

### 예측 및 매매

In [41]:
trainX_10_10, trainY_10_10 = make_train(10, 10, 'y')
testX_10_10, testY_10_10, lst_code_date_test = make_test(10, 10, 'y')

xgb_10_10_param = XGBClassifier(learning_rate=0.1, max_depth=4, n_estimators=500)
xgb_10_10_param.fit(trainX_10_10, trainY_10_10)
predY_prob = xgb_10_10_param.predict_proba(testX_10_10)
predY = xgb_10_10_param.predict(testX_10_10)
accuracy = accuracy_score(testY_10_10, predY)
print("xgb_10_10_param: ",accuracy)

/home/sookim06/.local/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[23:26:01] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
xgb_10_10_param:  0.7901812015921599


In [42]:
make_assignment4(100, 10)

### 수익률 계산

In [43]:
make_rate(100, 10)

9886it [00:45, 217.99it/s]

Final earning rate : 809.9887799999999 %


- 3차 모델에 비해 정확도 상승
- 수익률 증가 
- __최종 모델:__ 4차 모델 __(xgb_10_10_param)__

# 3. 결과

In [44]:
total_model = xgb_10_10_param

- 검증 데이터셋 

In [45]:
predY_prob = total_model.predict_proba(testX_10_10)
predY = total_model.predict(testX_10_10)
accuracy = accuracy_score(testY_10_10, predY)
print("total_model: ",accuracy)

total_model:  0.7901812015921599


- private 시험 데이터셋

In [36]:
IF=open("assignment3-2_private.txt",'r')
lst_code_date_test_private=[]
testX=[]
testY=[]
for line in IF:
    code, date, x, y = line.strip().split("\t")
    lst_code_date_test_private.append([code, date])
    testX.append(list(map(float, x.split(","))))
    testY.append(int(y))
testX=np.array(testX)
testY=np.array(testY)

In [37]:
predY_prob_private = total_model.predict_proba(testX)
predY_private = total_model.predict(testX)
accuracy = accuracy_score(testY, predY_private)
print("total_model_private: ",accuracy)

total_model_private:  0.8055854753513542


--------------------

- 제출 파일 생성

In [46]:
lst_output=[]
for (code, date), y in zip(lst_code_date_test, predY_prob):
    if y[1] >= 0.9: 
        lst_output.append([code, date, "buy", "all"])  
        lst_output.append([code, date+"n", "sell", "all"])  
    elif y[1] >= 0.7: 
        lst_output.append([code, date, "buy", "r40"])  
        lst_output.append([code, date+"n", "sell", "all"]) 
    elif y[1] >= 0.6: 
        lst_output.append([code, date, "buy", "r20"])  
        lst_output.append([code, date+"n", "sell", "all"])  
    elif y[1] >= 0.5: 
        lst_output.append([code, date, "buy", "r10"])  
        lst_output.append([code, date+"n", "sell", "all"]) 



lst_output.sort(key=lambda x:x[1]) 
OF=open("trading2022firsthalf.txt",'w') 
for row in lst_output:
    OF.write("\t".join(map(str, row))+"\n")
OF.close()

In [39]:
lst_output=[]
for (code, date), y in zip(lst_code_date_test_private, predY_prob_private):
    if y[1] >= 0.9: 
        lst_output.append([code, date, "buy", "all"])  
        lst_output.append([code, date+"n", "sell", "all"])  
    elif y[1] >= 0.7: 
        lst_output.append([code, date, "buy", "r40"])  
        lst_output.append([code, date+"n", "sell", "all"]) 
    elif y[1] >= 0.6: 
        lst_output.append([code, date, "buy", "r20"])  
        lst_output.append([code, date+"n", "sell", "all"])  
    elif y[1] >= 0.5: 
        lst_output.append([code, date, "buy", "r10"])  
        lst_output.append([code, date+"n", "sell", "all"]) 



lst_output.sort(key=lambda x:x[1]) 
OF=open("trading2022secondhalf.txt",'w') 
for row in lst_output:
    OF.write("\t".join(map(str, row))+"\n")
OF.close()

- 수익률 결과

In [48]:
start_money = 10000000 # 초기 현금 1천만원
money = start_money
dic_code2num ={}  # 보유 종목

IF=open("trading2022firsthalf.txt",'r')
for i, line in enumerate(tqdm(IF)): #주문 일지를 한 줄 읽어 옴
    code, date, request, amount = line.strip().split("\t")

    sql_query = '''
                SELECT *
                FROM stock_{}
                WHERE Date BETWEEN '2021-01-01' AND '2021-06-31'
                '''.format(code)
    stock = pd.read_sql(sql = sql_query, con = db_dsml)
    lst_data = stock.values.tolist()    

    ##############################################################################################
    for idx, row in enumerate(lst_data):
        data_date = row[0].date().strftime('%Y%m%d')


        if 'n' in date:
            date_n = date[:-1]
            if data_date == date_n: 
                close = lst_data[idx+1][-3] 
                break            

        else: 
            if data_date == date:
                close = row[-3] 
                break
    ##############################################################################################

    if request == 'buy': 
        if amount.startswith('r'):
            request_money = money * float(amount.lstrip("r")) / 100
        elif amount == 'all':
            request_money = money
        elif amount.isdigit():
            request_money = int(amount)
        else:
            raise Exception('Not permitted option')
        request_money = min(request_money, money)
        buy_num = int(request_money / close)
        money -= buy_num * close  
        if code not in dic_code2num:
            dic_code2num[code] = 0
        dic_code2num[code] += buy_num 
    if request == 'sell':
        if amount == 'all':
            sell_num = dic_code2num[code]
        else:
            raise Exception('Not permitted option')            
        money += sell_num * close
        dic_code2num[code] -= sell_num
        if dic_code2num[code] == 0:
            del dic_code2num[code]
IF.close()            

if dic_code2num != {}: 
    raise Exception('Not empty stock') 

print("Final earning rate : {} %".format(str((money-start_money) / start_money * 100)))
#     return str((money-start_money) / start_money * 100)

9886it [00:27, 364.92it/s]

Final earning rate : 809.9887799999999 %


In [40]:

start_money = 10000000 # 초기 현금 1천만원
money = start_money
dic_code2num ={}  # 보유 종목

IF=open("trading2022secondhalf.txt",'r')
for i, line in enumerate(tqdm(IF)): #주문 일지를 한 줄 읽어 옴
    code, date, request, amount = line.strip().split("\t")

    sql_query = '''
                SELECT *
                FROM stock_{}
                WHERE Date BETWEEN '2021-07-01' AND '2021-12-31'
                '''.format(code)
    stock = pd.read_sql(sql = sql_query, con = db_dsml)
    lst_data = stock.values.tolist()    

    ##############################################################################################
    for idx, row in enumerate(lst_data):
        data_date = row[0].date().strftime('%Y%m%d')


        if 'n' in date:
            date_n = date[:-1]
            if data_date == date_n: 
                close = lst_data[idx+1][-3] 
                break            

        else: 
            if data_date == date:
                close = row[-3] 
                break
    ##############################################################################################

    if request == 'buy': 
        if amount.startswith('r'):
            request_money = money * float(amount.lstrip("r")) / 100
        elif amount == 'all':
            request_money = money
        elif amount.isdigit():
            request_money = int(amount)
        else:
            raise Exception('Not permitted option')
        request_money = min(request_money, money)
        buy_num = int(request_money / close)
        money -= buy_num * close  
        if code not in dic_code2num:
            dic_code2num[code] = 0
        dic_code2num[code] += buy_num 
    if request == 'sell':
        if amount == 'all':
            sell_num = dic_code2num[code]
        else:
            raise Exception('Not permitted option')            
        money += sell_num * close
        dic_code2num[code] -= sell_num
        if dic_code2num[code] == 0:
            del dic_code2num[code]
IF.close()            

if dic_code2num != {}: 
    raise Exception('Not empty stock') 

print("Final earning rate : {} %".format(str((money-start_money) / start_money * 100)))
#     return str((money-start_money) / start_money * 100)

10104it [00:34, 291.00it/s]

Final earning rate : 504.20385999999996 %
